In [1]:
from PIL import Image, ImageDraw, ImageFont
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
import cv2
from matplotlib import pyplot as plt
import math
from sklearn import neighbors
import os
import os.path
import pickle
from sklearn import svm

In [2]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [4]:
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    """
    Trains a k-nearest neighbors classifier for face recognition.

    :param train_dir: directory that contains a sub-directory for each known person, with its name.

     (View in source code to see train_dir example tree structure)

     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...

    :param model_save_path: (optional) path to save model on disk
    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified
    :param knn_algo: (optional) underlying data structure to support knn.default is ball_tree
    :param verbose: verbosity of training
    :return: returns knn classifier that was trained on the given data.
    """
    X = []
    y = []

    # Loop through each person in the training set
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # Loop through each training image for the current person
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # Determine how many neighbors to use for weighting in the KNN classifier
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [22]:
def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.6):
    """
    Recognizes faces in given image using a trained KNN classifier

    :param X_img_path: path to image to be recognized
    :param knn_clf: (optional) a knn classifier object. if not specified, model_save_path must be specified.
    :param model_path: (optional) path to a pickled knn classifier. if not specified, model_save_path must be knn_clf.
    :param distance_threshold: (optional) distance threshold for face classification. the larger it is, the more chance
           of mis-classifying an unknown person as a known one.
    :return: a list of names and face locations for the recognized faces in the image: [(name, bounding box), ...].
        For faces of unrecognized persons, the name 'unknown' will be returned.
    """
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    # Load a trained KNN model (if one was passed in)
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    # Load image file and find face locations
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # If no faces are found in the image, return an empty result.
    if len(X_face_locations) == 0:
        return []

    # Find encodings for faces in the test iamge
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Use the KNN model to find the best matches for the test face
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=2)
    print(closest_distances)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Predict classes and remove classifications that aren't within the threshold
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]

In [20]:
def show_prediction_labels_on_image(img_path, predictions):
    """
    Shows the face recognition results visually.

    :param img_path: path to image to be recognized
    :param predictions: results of the predict function
    :return:
    """
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)
    font = ImageFont.load_default()
    for name, (top, right, bottom, left) in predictions:
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # # There's a bug in Pillow where it blows up with non-UTF-8 text
        # # when using the default bitmap font
        # name = name.encode("UTF-8")

        # # Draw a label with a name below the face
        # text_height = 10
        # draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        # draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))
        font = ImageFont.load_default()
        draw.text((0, 0),name, fill=(255,0,0), font=font)

    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
    display(pil_image)


In [7]:
knn_clf = train('./images', model_save_path="trained_knn_model.clf")

In [26]:
predictions = predict('./unknown-trump.jpeg', knn_clf=knn_clf)

(array([[0.53843062, 0.64654359]]), array([[2, 1]]))


In [1]:
predictions

NameError: name 'predictions' is not defined

In [ ]:
show_prediction_labels_on_image('./unknown-trump.jpeg', predictions)

In [32]:
face_images = {
    "trump": [
        "./images/trump/trump1.jpg"
    ],
    "ariana":[
        "./images/ariana/ariana1.jpg"
    ],
    "putin": [
        "./images/putin/putin1.jpg"
    ],
    "biden": [
        "./images/biden/biden1.jpg"
    ],
}

encodings = []
names = []

for person, image_paths in face_images.items():
    for path in image_paths:
        face = face_recognition.load_image_file(path)
        face_bbs = face_recognition.face_locations(face)
        if len(face_bbs) != 1:
            print(f"Can't analyze image at path {path} of {person}")
            continue
        face_encoding = face_recognition.face_encodings(face)[0]
        encodings.append(face_encoding)
        names.append(person)

# svm_clf = svm.LinearSVC()
svm_clf = svm.SVC(probability=True, gamma='scale')
svm_clf.fit(encodings, names)

SVC(probability=True)

In [44]:
test_image = face_recognition.load_image_file('./anime.jpg')
test_face_locations = face_recognition.face_locations(test_image)
test_face_locations
test_face_encoding = face_recognition.face_encodings(test_image, known_face_locations=test_face_locations)
test_face_encoding

[(222, 696, 407, 510)]

In [3]:
test_image = face_recognition.load_image_file('./images/trump/trump1.jpg')
test_face_locations = face_recognition.face_locations(test_image)
test_face_encoding = face_recognition.face_encodings(test_image)[0]
test_face_encoding

array([-0.15269653,  0.10987215,  0.02744502,  0.01149364, -0.12665196,
       -0.03977503,  0.04482668, -0.17497881,  0.0678868 , -0.08996343,
        0.18113542, -0.08842672, -0.36437851, -0.07972416,  0.0031369 ,
        0.12855591, -0.13808133, -0.15942805, -0.20948088, -0.12257048,
        0.03341122,  0.00479252, -0.03267508, -0.03947413, -0.10862232,
       -0.22754619, -0.05384813, -0.13053878, -0.00486688, -0.09233195,
        0.06154098, -0.00275521, -0.20462567, -0.12419596, -0.00184974,
       -0.00778652, -0.09120831, -0.06746903,  0.18022443, -0.02576419,
       -0.16877732, -0.02345786,  0.03031982,  0.20937589,  0.21699429,
        0.01485616, -0.00997108, -0.15936315,  0.10122295, -0.28451821,
       -0.01571452,  0.14753629,  0.16091141,  0.11327104,  0.06429446,
       -0.1358415 ,  0.05266336,  0.13518067, -0.22139801,  0.06652847,
        0.0726627 , -0.16993606, -0.03742438, -0.04675952,  0.11315753,
        0.07875495, -0.01854436, -0.1279642 ,  0.27543205, -0.17

In [23]:
list(test_face_encoding)

[-0.1251586377620697,
 0.10279892385005951,
 0.08014370501041412,
 -0.12285250425338745,
 -0.167522594332695,
 -0.08154484629631042,
 -0.13406755030155182,
 -0.09916097670793533,
 0.17467071115970612,
 -0.1265711784362793,
 0.2022881805896759,
 -0.02566657029092312,
 -0.22026726603507996,
 -0.07307475060224533,
 -0.024724824354052544,
 0.276022732257843,
 -0.18145090341567993,
 -0.1534252166748047,
 -0.019235659390687943,
 -0.06835337728261948,
 0.0388641394674778,
 0.02689986303448677,
 0.015718869864940643,
 0.11896955966949463,
 -0.1386169195175171,
 -0.36444219946861267,
 -0.0946066603064537,
 -0.11590753495693207,
 -0.17955169081687927,
 -0.024422235786914825,
 -0.059644460678100586,
 0.02391854301095009,
 -0.22474715113639832,
 -0.07375185936689377,
 0.08006671071052551,
 0.1253444105386734,
 -0.0012762227561324835,
 -0.08967531472444534,
 0.11196378618478775,
 -0.0031091044656932354,
 -0.2187918871641159,
 -0.01810726523399353,
 0.17381441593170166,
 0.1827765703201294,
 0.19378

In [6]:
comparing_face = face_recognition.load_image_file('./images/huou/huou.jpg')
comparing_face_locations = face_recognition.face_locations(comparing_face)
comparing_face_encoding = face_recognition.face_encodings(comparing_face)[0]
comparing_face_encoding

array([-1.53846055e-01,  6.13332242e-02,  5.18904701e-02, -1.24199219e-01,
       -1.16682023e-01, -1.00067154e-01, -1.05487488e-01, -6.65155649e-02,
        1.72323138e-01, -1.38931900e-01,  1.95000827e-01, -2.45214347e-02,
       -2.46265769e-01, -8.12552199e-02, -2.96929833e-02,  2.86456198e-01,
       -1.71053812e-01, -1.33136079e-01, -3.54465283e-02, -4.07256484e-02,
        8.79964381e-02,  3.25876400e-02,  5.08354604e-02,  1.03533037e-01,
       -1.23551592e-01, -3.72065008e-01, -1.24075949e-01, -8.34776536e-02,
       -1.07404530e-01, -2.86892280e-02, -6.64995164e-02,  1.39128836e-02,
       -2.29611740e-01, -6.03579432e-02,  5.35075925e-02,  9.99522060e-02,
        8.65654074e-05, -1.16375111e-01,  1.44729450e-01,  4.94986437e-02,
       -2.42517188e-01, -4.53049727e-02,  1.28711149e-01,  1.79457396e-01,
        1.66822866e-01,  4.22745571e-02,  2.34570932e-02, -5.58876432e-02,
        1.27249390e-01, -2.62346476e-01, -9.75659769e-03,  1.08664043e-01,
        1.64192896e-02,  

In [11]:
import numpy as np
face_recognition.compare_faces([test_face_encoding], np.array([-1.53846055e-01,  6.13332242e-02,  5.18904701e-02, -1.24199219e-01,
       -1.16682023e-01, -1.00067154e-01, -1.05487488e-01, -6.65155649e-02,
        1.72323138e-01, -1.38931900e-01,  1.95000827e-01, -2.45214347e-02,
       -2.46265769e-01, -8.12552199e-02, -2.96929833e-02,  2.86456198e-01,
       -1.71053812e-01, -1.33136079e-01, -3.54465283e-02, -4.07256484e-02,
        8.79964381e-02,  3.25876400e-02,  5.08354604e-02,  1.03533037e-01,
       -1.23551592e-01, -3.72065008e-01, -1.24075949e-01, -8.34776536e-02,
       -1.07404530e-01, -2.86892280e-02, -6.64995164e-02,  1.39128836e-02,
       -2.29611740e-01, -6.03579432e-02,  5.35075925e-02,  9.99522060e-02,
        8.65654074e-05, -1.16375111e-01,  1.44729450e-01,  4.94986437e-02,
       -2.42517188e-01, -4.53049727e-02,  1.28711149e-01,  1.79457396e-01,
        1.66822866e-01,  4.22745571e-02,  2.34570932e-02, -5.58876432e-02,
        1.27249390e-01, -2.62346476e-01, -9.75659769e-03,  1.08664043e-01,
        1.64192896e-02,  4.55213897e-02,  6.60196245e-02, -1.05020642e-01,
        1.38697088e-01,  1.48665458e-01, -2.61562914e-01, -6.84359446e-02,
        5.84008023e-02, -1.22998923e-01, -7.85408616e-02, -2.46204715e-02,
        3.40129524e-01,  1.87817737e-01, -1.40786424e-01, -1.34534106e-01,
        1.76187173e-01, -1.27364993e-01, -4.50569429e-02,  1.60727948e-02,
       -1.50385126e-01, -1.63846314e-01, -3.37438315e-01, -5.46752848e-03,
        3.64943594e-01,  1.35758728e-01, -1.38977855e-01,  1.24585398e-01,
       -6.49083704e-02,  4.58826013e-02,  6.51599318e-02,  1.23794496e-01,
       -2.13619284e-02,  7.94594660e-02, -9.13027152e-02,  2.58933175e-02,
        1.80924818e-01, -2.47490164e-02,  2.84806341e-02,  2.57637590e-01,
        9.77712777e-03,  7.56968856e-02,  6.69860318e-02,  4.39015701e-02,
       -1.02937646e-01, -8.79815072e-02, -1.30717888e-01,  5.05617214e-03,
        3.34874690e-02, -1.32932356e-02, -1.34143997e-02,  8.33467171e-02,
       -1.71651468e-01,  4.25577164e-02, -4.19344828e-02, -5.65714538e-02,
       -8.48146155e-02, -2.85350555e-03, -7.19453916e-02, -7.13668466e-02,
        1.54125635e-02, -2.20401332e-01,  2.04973042e-01,  1.51466087e-01,
       -1.90525036e-02,  1.49220988e-01,  7.73977339e-02,  5.69722094e-02,
       -5.85370027e-02, -8.08175802e-02, -1.44082919e-01, -7.76207596e-02,
        1.29951209e-01, -4.55829501e-02,  1.39252737e-01,  6.62557222e-03]))

[True]

In [30]:
multiple_people_image = face_recognition.load_image_file('./multiple-people.jpg')
multiple_people_face_locations = face_recognition.face_locations(multiple_people_image)
face_features = face_recognition.face_encodings(multiple_people_image, known_face_locations=multiple_people_face_locations)
# for face_bb in multiple_people_face_locations:
#     top, right, bottom, left = face_bb
#     print(face_features)
face_features
svm_clf.predict(face_features)

array([[0.20796845, 0.19955125, 0.19751443, 0.19787987, 0.19708601],
       [0.20837915, 0.20013739, 0.20409076, 0.19272866, 0.19466404],
       [0.20770625, 0.20040839, 0.19260172, 0.20210602, 0.19717762]])

In [6]:
with open('../models/svm_model.pkl', 'wb') as f:
    pickle.dump(svm_clf, f)

In [7]:
with open('../models/svm_model.pkl', 'rb') as f:
    saved_svm_clf = pickle.load(f)
    

In [9]:
print(saved_svm_clf.predict([test_face_encoding]))


['biden']


In [ ]:
svm_clf.fit